In [ ]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasetsS
import torchvision.transforms as transforms
import torchvision
from customDatasets import CatsAndDogsDataset

In [ ]:
# creted normal pass neural network
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [ ]:
# select device
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

In [ ]:
# hyperparameters
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

In [ ]:
# prparing custom datasets
# import pandas as pd
# import os
# import numpy as np
# allList = os.listdir(r'./cats_dogs_data/train')
# catList = [i for i in allList if i[0:3]=='cat']
# print(len(catList))
# dogList = np.array(list(set(allList)-set(catList)))
# print(type(dogList))

# catLabel = np.zeros(len(catList))
# dogLabel = np.ones(len(dogList))

# path = np.append(catList,dogList)
# label = np.append(catLabel,dogLabel).astype('int')

# data = pd.DataFrame({'path':path, 'label':label})
# data.to_csv(r'./cats_dogs_data/train.csv', index=False)

In [ ]:
# building custom datasets
dataset = CatsAndDogsDataset('your_csv', 
                             'folder_path', 
                             transform=transforms.Compose(
                                 [transforms.ToTensor(), 
                                  transforms.Resize((224,224))]))

train_set, test_set = torch.utils.data.random_split(
                                                    dataset=dataset, 
                                                    lengths=[int(0.8*len(dataset)), int(0.2*len(dataset))]
                                                   )

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)                      
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

In [ ]:
# initialize network
model = torchvision.models.vgg16(pretrained=True,)
model.features[0] = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), )
model.avgpool = Identity()
model.classifier[6] = nn.Linear(in_features=4096, out_features=2, bias=True)
model = model.to(device)

In [ ]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# train network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # get data to cuda if possible
        data = data.to(device=device)           # for FCN, CNN and RNN
        # data = data.to(device=device).squeeze(1)  # for GRU
        
        targets = targets.to(device=device)
        # get to correct shape
        # data = data.reshape(data.shape[0], -1)  # for FCN 
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient or adam step
        optimizer.step()

In [ ]:
# check accuracy in training and test to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device = device)            
            y = y.to(device = device)
            
            scores = model(x)
            
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        print(f'accuracy is {float(num_correct)*100/float(num_samples):.2f}')
    
    model.train()

In [ ]:
check_accuracy(test_loader, model)